In [1]:
import csv
import time
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

In [2]:
file = open("Activity_clean.csv", "w", encoding="utf8")
f = csv.writer(file)
f.writerow(["Userid", "senseStartTimeMillis", "senseStartTime", "Activity", "Survey Date", "Final Date", "Time", "ADHD"])

86

In [3]:
df = pd.read_csv("weekly_data_for_visualization.csv")
ids = df["Q317"].unique()
ids=ids[:-2] #only keeping the numbers of ids

In [4]:
# dictionary of ADHD labels for each user id
ADHD_dict = {}
baseline = pd.read_csv("sassema baseline 3.16.2023.csv")
baseline['ID'] = baseline['ID'].fillna(0)

for index in baseline.index:
    if index > 1:
        if baseline["ID"][index] != 0:
            if baseline["ID"][index] == "773":
                baseline["ID"][index] = "773684"
            if baseline["ADHDSx_Dx"][index] == "No":
                ADHD_dict[int(baseline["ID"][index])] = 0
            elif baseline["ADHDSx_Dx"][index] == "Yes":
                ADHD_dict[int(baseline["ID"][index])] = 1

In [5]:
# create a list of dates from mili sec
def changeDateType(dates):
    date_final = [i.split(" ")[0] for i in dates]
    converted_dates = []
    for i in range(0,len(date_final)):  
        date_object = datetime.strptime(date_final[i], '%m/%d/%Y')
        converted_dates.append(date_object)
    return converted_dates

In [6]:
def changeYYMMDD(date):
    index = date.index("-")
    date = date[index-4:index+6]
    date = date.replace("-","/")
    date_object = datetime.strptime(date, '%Y/%m/%d')

    return date_object

In [7]:
def addBaseLine(idf,surveyKeep):
    df = pd.read_csv("sassema baseline 3.16.2023.csv")

    if idf == "773684":
        idf = "773"
    
    baselineDate = str(df.loc[df["ID"]==idf, 'StartDate'])

    date_object = changeYYMMDD(baselineDate)

    surveyKeep.insert(0,date_object)

In [8]:
def addFinalDate():
    de = pd.read_csv("Activity_joined.csv")
    dates_milli = de[" senseStartTime"].tolist()
    date_mili_strip = [i.replace("'", "") for i in dates_milli]
    date_final = [i.split("T")[0] for i in date_mili_strip]

    converted_dates = []

    for i in date_final:  # create a list fo dates from mili sec
        converted_dates.append(i)

    de["Final Date"] = converted_dates  # new conlumn of dates only
    de.to_csv("Activity_joined.csv", index=False)

In [9]:
Userid = [] 
senseStartTimeMillis = [] 
senseStartTime = [] 
Activiy = [] 
SurveyDate = []
FinalDate = []
ADHD = []

threshold = 4
addFinalDate()
de = pd.read_csv("Activity_joined.csv")
for id in ids:
    # dates
    surveydates = df.loc[df["Q317"]==id, 'StartDate'] 
    # add the survey dates of a user id to a list
    surveyKeep = changeDateType(surveydates)

    addBaseLine(id,surveyKeep)
    
    # DECIDE WHAT SURVEY DATES ARE KEPT 
    i = 1
    # discard current survey if the time interval between previous and current survey is not 7 
    while i < len(surveyKeep):

        numDays = surveyKeep[i] - surveyKeep[i-1]
        strDays = str(numDays)

        difference = strDays.split(" ")[0]
        difference = int(difference)

        if difference < 7:
            surveyKeep.pop(i) 

        else: 
            i = i + 1 
    
    # DECIDE WHAT SENING DATES ARE KEPT BASED ON TRESHOLD 
    
    
    
    #create a list of sensing data

    sensingDates = de.loc[(de["Userid"])==int(id), 'Final Date'] 
    sensingKeep = []

    if len(sensingDates) != 0:
        for d in sensingDates:
            date_object = changeYYMMDD(d)
            if date_object not in sensingKeep:
                sensingKeep.append(date_object)
    count = 0
    for i in range(1,len(surveyKeep)):
        if (len(sensingKeep) != 0):
            count = 0

            for j in range(0,len(sensingKeep)):
                if surveyKeep[i] == sensingKeep[j]:
                    numDiff = 0
                else: 
                    numDiff = surveyKeep[i] - sensingKeep[j]
                    numDiff = str(numDiff)
                    numDiff = numDiff.split(" ")[0]
                    numDiff = int(numDiff)
                if (numDiff < 7) and (numDiff >= 0):
                    count += 1
                    
        id_df = de.loc[(de["Userid"])==int(id)]  
        id_df.to_csv("temp.csv", index=False)
        id_de = pd.read_csv("temp.csv")
        
#         print(surveyKeep[i])
        stringSurvey = str(surveyKeep[i])
        stringSurvey = stringSurvey.split(" ")[0]
#         print(stringSurvey)
        
        if (count >= threshold):
            # iterate through the final date column when Userid=id
            for k in range(len(id_de["Final Date"])):
#                 print(k)
                date = id_de.loc[id_de.index[k], 'Final Date']
                dateObject = changeYYMMDD(date)
#                 print(id, date)
                if surveyKeep[i] == dateObject:
                    numDiff = 0
                else:
                    numDiff = surveyKeep[i] - dateObject
                    numDiff = str(numDiff)
                    numDiff = numDiff.split(" ")[0]
                    numDiff = int(numDiff)
                
                # if final date is within 7 day interval from survey date
                if (numDiff < 7) and (numDiff >= 0):
                    # append to all lists
                    Userid.append(id_de.loc[k, 'Userid']) 
                    senseStartTimeMillis.append(id_de.loc[k, ' senseStartTimeMillis'])  
                    senseStartTime.append(id_de.loc[k, ' senseStartTime'])
                    Activiy.append(id_de.loc[k, ' Activity'])
                    SurveyDate.append(stringSurvey)
                    FinalDate.append(id_de.loc[k, 'Final Date'])
                    ADHD.append(ADHD_dict[id_de.loc[k, 'Userid']])

        

In [10]:
for l in range(len(Userid)):
    FinalDate[l] = FinalDate[l][1:]
    index = senseStartTime[l].index("T")
    time = senseStartTime[l][(index+1):]
    f.writerow([Userid[l], senseStartTimeMillis[l], senseStartTime[l], Activiy[l], '"' + SurveyDate[l] + '"','"' + FinalDate[l] + '"','"' + time + '"',ADHD[l]])

file.close()

In [11]:
# sort the values in the csv file by Userid and time of activity 
df = pd.read_csv("Activity_clean.csv")

df = df.sort_values(by=['Userid','senseStartTimeMillis'],ascending=[True,True])
df.to_csv('Activity_clean.csv',index=False)